In [1]:
!git clone https://github.com/kimbold/VRDL_2019

Cloning into 'VRDL_2019'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 3978 (delta 23), reused 106 (delta 19), pack-reused 3865
Receiving objects: 100% (3978/3978), 55.12 MiB | 27.16 MiB/s, done.
Resolving deltas: 100% (23/23), done.


**Load svhn dataset**

In [2]:
%cd VRDL_2019/HW3

/content/VRDL_2019/HW3


**Import required modules**

In [3]:
import numpy as np
np.random.seed(111)
import argparse
import os
import json
from yolo.frontend import create_yolo, get_object_labels

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


**Train yolo**

In [0]:
#Train and validate YOLO_v2 model on any dataset
def setup_training(config_file):
    """make directory to save weights & its configuration """
    import shutil
    with open(config_file) as config_buffer:
        config = json.loads(config_buffer.read())
    dirname = config['train']['saved_folder']
    if os.path.isdir(dirname):
        print("{} is already exists. Weight file in directory will be overwritten".format(dirname))
    else:
        print("{} is created.".format(dirname, dirname))
        os.makedirs(dirname)
    print("Weight file and Config file will be saved in \"{}\"".format(dirname))
    shutil.copyfile(config_file, os.path.join(dirname, "config.json"))
    return config, os.path.join(dirname, "weights.h5")


def train(conf="configs/from_scratch.json"):
  #path to configuration file

    config, weight_file = setup_training(conf)
    
    if config['train']['is_only_detect']:
        labels = ["object"]
    else:
        if config['model']['labels']:
            labels = config['model']['labels']
        else:
            labels = get_object_labels(config['train']['train_annot_folder'])
    print(labels)

    # 1. Construct the model 
    yolo = create_yolo(config['model']['architecture'],
                       labels,
                       config['model']['input_size'],
                       config['model']['anchors'],
                       config['model']['coord_scale'],
                       config['model']['class_scale'],
                       config['model']['object_scale'],
                       config['model']['no_object_scale'])
    
    # 2. Load the pretrained weights (if any) 
    yolo.load_weights(config['pretrained']['full'], by_name=True)

    # 3. actual training 
    yolo.train(config['train']['train_image_folder'],
               config['train']['train_annot_folder'],
               config['train']['actual_epoch'],
               weight_file,
               config["train"]["batch_size"],
               config["train"]["jitter"],
               config['train']['learning_rate'], 
               config['train']['train_times'],
               config['train']['valid_times'],
               config['train']['valid_image_folder'],
               config['train']['valid_annot_folder'],
               config['train']['first_trainable_layer'],
               config['train']['is_only_detect'])
    # loss: 2.1691, train batch jitter=False


In [5]:
#!python train.py -c configs/from_scratch2.json # Alternative training 
train()

svhn is created.
Weight file and Config file will be saved in "svhn"
['object']














/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 422, 422, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 208, 208, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 208, 208, 64) 256         conv1[0][0]                      
___________________________

In [11]:
train(conf="configs/from_scratch2.json")

svhn is already exists. Weight file in directory will be overwritten
Weight file and Config file will be saved in "svhn"
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 422, 422, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 208, 208, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 208, 208, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

**Evaluate trained yolo digit detector**

In [0]:
#!python evaluate.py -c configs/from_scratch.json -w svhn/weights.h5

In [7]:
import argparse
import json
import cv2
import numpy as np
from yolo.frontend import create_yolo
from yolo.backend.utils.box import draw_scaled_boxes
from yolo.backend.utils.annotation import parse_annotation
from yolo.backend.utils.eval._box_match import BoxMatcher

import os
import yolo

DEFAULT_CONFIG_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "config.json")
DEFAULT_WEIGHT_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "weights.h5")
DEFAULT_THRESHOLD = 0.3


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [0]:
def count_true_positives(detect_boxes, true_boxes, detect_labels=None, true_labels=None):
    """
    # Args
        detect_boxes : array, shape of (n_detected_boxes, 4)
        true_boxes : array, shape of (n_true_boxes, 4)
        detected_labels : array, shape of (n_detected_boxes,)
        true_labels :
    """
    n_true_positives = 0
    print("Detect boxes:",detect_boxes,"True boxes", true_boxes)
    matcher = BoxMatcher(detect_boxes, true_boxes, detect_labels, true_labels)
    for i in range(len(detect_boxes)):
        matching_idx, iou = matcher.match_idx_of_box1_idx(i)
        # print("detect_idx: {}, true_idx: {}, matching-score: {}".format(i, matching_idx, iou))
        if matching_idx is not None and iou > 0.5:
            n_true_positives += 1
    return n_true_positives


def calc_score(n_true_positives, n_truth, n_pred):
    """
    # Args
        detect_boxes : list of box-arrays
        true_boxes : list of box-arrays
    """
    print("True positives:", n_true_positives)
    print("n_pred:", n_pred)
    print("n_truth:", n_truth)
    precision = n_true_positives / n_pred
    recall = n_true_positives / n_truth
    print("Precision:", precision)
    print("Recall:", recall)
    fscore = 2* precision * recall / (precision + recall)
    score = {"fscore": fscore, "precision": precision, "recall": recall}
    return score

In [0]:
def evaluate(conf=DEFAULT_CONFIG_FILE, weights=DEFAULT_WEIGHT_FILE, threshold=DEFAULT_THRESHOLD):

      with open(conf) as config_buffer:
          config = json.loads(config_buffer.read())

      # 2. create yolo instance & predict
      yolo = create_yolo(config['model']['architecture'],
                        config['model']['labels'],
                        config['model']['input_size'],
                        config['model']['anchors'])
      yolo.load_weights(weights)

      # 3. read image
      write_dname = "detected"
      if not os.path.exists(write_dname): os.makedirs(write_dname)
      annotations = parse_annotation(config['train']['valid_annot_folder'],
                                    config['train']['valid_image_folder'],
                                    config['model']['labels'],
                                    is_only_detect=config['train']['is_only_detect'])

      n_true_positives = 0
      n_truth = 0
      n_pred = 0
      for i in range(len(annotations)):
          img_path = annotations.fname(i)
          img_fname = os.path.basename(img_path)
          image = cv2.imread(img_path)
          true_boxes = annotations.boxes(i)
          true_labels = annotations.code_labels(i)
          
          boxes, probs = yolo.predict(image, float(threshold))
          labels = np.argmax(probs, axis=1) if len(probs) > 0 else [] 
        
          # 4. save detection result
          image = draw_scaled_boxes(image, boxes, probs, config['model']['labels'])
          output_path = os.path.join(write_dname, os.path.split(img_fname)[-1])
          
          cv2.imwrite(output_path, image)
          print("{}-boxes are detected. {} saved.".format(len(boxes), output_path))

          n_true_positives += count_true_positives(boxes, true_boxes, labels, true_labels)
          n_truth += len(true_boxes)
          n_pred += len(boxes)
      print(calc_score(n_true_positives, n_truth, n_pred))

In [12]:
evaluate()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 422, 422, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 208, 208, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 208, 208, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [0]:
def get_name(index, hdf5_data):
    name = hdf5_data['/digitStruct/name']
    return ''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value])

def get_bbox(index, hdf5_data):
    attrs = {}
    item = hdf5_data['digitStruct']['bbox'][index].item()
    for key in ['label', 'left', 'top', 'width', 'height']:
        attr = hdf5_data[item][key]
        values = [hdf5_data[attr.value[i].item()].value[0][0]
                  for i in range(len(attr))] if len(attr) > 1 else [attr.value[0][0]]
        attrs[key] = values
    return attrs